# Google Drive

本 Notebook 将指导您如何将 LangChain 连接到 `Google Drive API`。

## 先决条件

1. 创建一个 Google Cloud 项目或使用现有项目
1. 启用 [Google Drive API](https://console.cloud.google.com/flows/enableapi?apiid=drive.googleapis.com)
1. [授权桌面应用的凭据](https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application)
1. `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib`

## 获取 Google Docs 数据的说明
默认情况下，`GoogleDriveTools` 和 `GoogleDriveWrapper` 期望 `credentials.json` 文件位于 `~/.credentials/credentials.json`，但这可以通过将 `GOOGLE_ACCOUNT_FILE` 环境变量设置为您的 `custom/path/to/credentials.json` 来进行配置。
`token.json` 的位置使用相同的目录（或使用参数 `token_path`）。请注意，`token.json` 将在您首次使用该工具时自动创建。

`GoogleDriveSearchTool` 可以根据某些请求检索文件。

默认情况下，如果您使用 `folder_id`，则该文件夹内的所有文件都可以检索到 `Document`，前提是文件名与查询匹配。

In [ ]:
%pip install --upgrade --quiet  google-api-python-client google-auth-httplib2 google-auth-oauthlib langchain-community

您可以通过 URL 获取您的文件夹和文档 ID：

* 文件夹：https://drive.google.com/drive/u/0/folders/1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5 -> 文件夹 ID 是 `"1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5"`
* 文档：https://docs.google.com/document/d/1bfaMQ18_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw/edit -> 文档 ID 是 `"1bfaMQ18_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw"`

特殊值 `root` 指的是您的个人根目录。

In [1]:
folder_id = "root"
# folder_id='1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5'

默认情况下，所有具有以下 MIME 类型的文件的都可以转换为 `Document`。
- text/text
- text/plain
- text/html
- text/csv
- text/markdown
- image/png
- image/jpeg
- application/epub+zip
- application/pdf
- application/rtf
- application/vnd.google-apps.document (GDoc)
- application/vnd.google-apps.presentation (GSlide)
- application/vnd.google-apps.spreadsheet (GSheet)
- application/vnd.google.colaboratory (Notebook colab)
- application/vnd.openxmlformats-officedocument.presentationml.presentation (PPTX)
- application/vnd.openxmlformats-officedocument.wordprocessingml.document (DOCX)

可以更新或自定义此设置。请参阅 `GoogleDriveAPIWrapper` 的文档。

但是，必须安装相应的软件包。

In [ ]:
%pip install --upgrade --quiet  unstructured langchain-googledrive

In [ ]:
import os

from langchain_googledrive.tools.google_drive.tool import GoogleDriveSearchTool
from langchain_googledrive.utilities.google_drive import GoogleDriveAPIWrapper

os.environ["GOOGLE_ACCOUNT_FILE"] = "custom/path/to/credentials.json"

# By default, search only in the filename.
tool = GoogleDriveSearchTool(
    api_wrapper=GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=2,
        template="gdrive-query-in-folder",  # Search in the body of documents
    )
)

In [3]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
tool.run("machine learning")

In [5]:
tool.description

"A wrapper around Google Drive Search. Useful for when you need to find a document in google drive. The input should be formatted as a list of entities separated with a space. As an example, a list of keywords is 'hello word'."

## 在 ReAct 代理中使用工具

为了创建一个使用 Google Jobs 工具的代理，请安装 Langgraph

In [ ]:
%pip install --upgrade --quiet langgraph langchain-openai

并使用 `create_react_agent` 功能来初始化 ReAct 代理。您还需要设置您的 OPEN_API_KEY（请访问 https://platform.openai.com）来访问 OpenAI 的聊天模型。

In [ ]:
import os

from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"


llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0)
agent = create_react_agent(llm, tools=[tool])

events = agent.stream(
    {"messages": [("user", "Search in google drive, who is 'Yann LeCun' ?")]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()